In [1]:
import spacy
import pandas as pd

In [2]:
nlp = spacy.load("fr_core_news_md")

In [3]:
#Lit le fichier des gares et filtrent sur les gares qui accueillent des voyageurs
liste_gare = pd.read_csv("./liste-des-gares.csv", sep=';')
liste_gare = liste_gare.loc[liste_gare["VOYAGEURS"] == "O"]

#Lit le jeu de tests
liste_phrase_test = pd.read_csv("./liste-de-phrases.csv", sep=";")
liste_phrase_test["étapes"] = liste_phrase_test["étapes"].fillna("") #remplace les NaN par une string vide, ça permet d'éviter des erreurs

#Applique des filtres sur la liste des gares pour une analyse approfondie plus tard par l'ia
gares = list(liste_gare["COMMUNE"])
gares = [x.lower() for x in gares]
gares_without_starting = [x[2:] for x in gares if x[0:2] == "l'"]
gares_without_dash = [x.replace("-", " ") for x in gares if "-" in x]
gares = [x[2:] if x[0:2] == "l'" else x for x in gares]

In [4]:
gares_without_starting

['hermitage',
 'isle-jourdain',
 "hospitalet-pres-l'andorre",
 'etang-la-ville',
 'aigle',
 'isle-sur-le-doubs',
 'escarene',
 'isle-sur-la-sorgue',
 'etang-la-ville',
 "isle-d'abeau",
 'argentiere-la-bessee',
 'etang-la-ville',
 'hopital-du-grosbois']

In [5]:
#Liste des prépositions et verbe nécessaire à l'analyse sémantique
preposition_depart = ["de", "depuis", "à", "sur", "dans", "vers"]
preposition_destination = ["à", "pour", "sur", "dans", "jusque", "jusqu'à", "vers", "de"]
preposition_in_both = ["de", "à", "sur", "dans", "vers"]
preposition_etape = ["par", "via", "puis", "et"]
preposition = set(preposition_depart + preposition_destination + preposition_etape)

verbe_action = ["aller", "partir", "rendre", "rejoindre", "retourner"]
verbe_potentiel_action = ["vouloir", "souhaiter", "essayer", "devoir", "pouvoir", "aimer"]
verbe_etapes = ["passer"]

In [6]:
#Liste de la ponctuation
ponctuation_list = [".", ",", ";", ":", "!", "?"]

In [15]:
def data_treatment(text):
    #Prétraitement des données pour une meilleur analyse
    text = text.lower()

    #On retire toute ponctuation parce qu'elles ne sont pas utiles
    for ponctu in ponctuation_list:
        text = text.replace(ponctu, "")
    
    #On recherche les noms de gares qui ont un nom qui peut poser des problèmes 
    for city in gares_without_dash:
        if city in text:
           text = text.replace(city, city.replace(" ", "-"))
    
    return text

In [19]:
text = "Pendant mon séjour à Paris, j'aimerais visiter Strasbourg. Peux tu me trouver un itinéraire ?"
def get_main_data(text):
    """
        Retourne une analyse sémantique de la phrase passé en paramètre, elle retourne un array contenant:
            - un array avec les labels de chaque mot de la forme [mot, label]
            - la première ville trouvée avec la préposition qui l'accompagne si trouvé: [ville] ou [préposition, ville]
            - la deuxième ville trouvée avec la préposition qui l'accompagne si trouvé: [ville] ou [préposition, ville]
    """
    #Initialisation des variables
    doc = nlp(text)
    position = []
    locs = []

    #Analyse sémantique de la phrase
    for w in doc:
        w_text = w.text
        if w.pos_ == "ADP" and w_text in preposition:
            position.append([w_text, w.pos_])
        if w_text in gares:
            #Pour une gare, on cherche à récupérer la préposition qui vient avant s'il y en a une et ensuite on ajoute nos résultats à locs et position
            tmp_loc = {
                "ADP": "",
                "LOC": ""
            }
            if position and position[-1][1] == "ADP":
                tmp_loc["ADP"] = position[-1][0]
            tmp_loc["LOC"] = w_text.upper()
            locs.append(tmp_loc)
            position.append([w_text.upper(), "LOC"])
        if w.pos_ == "VERB" and w.lemma_ in verbe_action:
            position.append([w.lemma_, "ACTION"])
        if w.pos_ == "VERB" and w.lemma_ in verbe_potentiel_action:
            position.append([w.lemma_, "POT_ACTION"])
    return [position, locs]
#print([(w.text, w.label_) for w in doc.ents])
text = data_treatment(text)
get_main_data(text)

[[['à', 'ADP'],
  ['PARIS', 'LOC'],
  ['aimer', 'POT_ACTION'],
  ['STRASBOURG', 'LOC'],
  ['pouvoir', 'POT_ACTION']],
 [{'ADP': 'à', 'LOC': 'PARIS'}, {'ADP': '', 'LOC': 'STRASBOURG'}]]

In [20]:
def is_departure(loc, loc_2):
    """
        vérifie si une ville est la ville de départ dans une phrase
    """
    if loc["ADP"] != "":
        if loc["ADP"] not in preposition_in_both:
            if loc["ADP"] in preposition_depart:
                return True
    elif loc["ADP"] == "" and loc_2["ADP"] == "":
        return True
    return False

In [21]:
def get_city_name(loc_1, loc_2):
    """
        Récupère le nom de la ville dans un array qui peut prendre la forme [Préposition, ville] ou [ville] 
    """
    return [loc_1["LOC"], loc_2["LOC"]]

In [22]:
def is_de_a(loc_1, loc_2):
    """
        Vérifie si on se retrouve dans le cas d'une phrase avec 2 villes, et les prépositions "de, à" devant le nom des villes
    """
    prep1 = loc_1["ADP"]
    prep2 = loc_2["ADP"]
    if prep1 in ["de", "à"] and prep2 in ["de", "à"]:
        return True
    return False

In [25]:
def get_position(text):
    """
        Retourne les villes trouvées dans le sens départ, destination ou une erreur si ce n'est pas possible
    """
    text = data_treatment(text)
    #On fait une première analyse sémantique pour récupérer des infos essentielles, voir fonction get_main_data() 
    [data, locs] = get_main_data(text)

    #Actuellement, si on a pas 2 villes on retourne une erreur
    if len(locs) <= 1:
        return ["Il manque des villes ou des villes n'ont pas été détecté. Il se peut aussi que la ville n'a pas été détecté parce qu'il n'y a pas de gare"]
    
    depart, destination, etapes = "", "", []

    #On commence par chercher les étapes en premier
    for city in locs:
        if city["ADP"] in preposition_etape:
            etapes.append(city["LOC"])
            locs.remove(city)
    
    if len(locs) == 2:
        loc_1, loc_2 = locs
    
        #if, elif permettent de chercher des cas simples sémantiquement et gagner du temps de travail, pour les cas plus complexes voir le else
        if is_departure(loc_1, loc_2):
            depart = loc_1["LOC"]
            destination = loc_2["LOC"]
        elif is_departure(loc_2, loc_1):
            depart = loc_2["LOC"]
            destination = loc_1["LOC"]
        elif is_de_a(loc_1, loc_2):
            if loc_1["ADP"] == "de":
                depart = loc_1["LOC"]
                destination = loc_2["LOC"]
            else:
                depart = loc_2["LOC"]
                destination = loc_1["LOC"]
        else:
            pot_action = False
            action = False
            
            #Analyse est divisé en 2 partie, la première est ce que j'appelle 
            # le potentiel d'action ou pot_action, qui a pour but d'analyser l'intention d'une phrase à l'aide de sa sémantique
            # par exemple: je souhaite aller à Paris, grâce au verbe souhaiter, on sait que la personne "souhaite"(à l'intention de) se rendre à Paris
            for [w, label] in data:
                if label == "POT_ACTION":
                    pot_action = True
                    pass
                
                if pot_action:
                    if label == "LOC":
                        destination = w
                        cities = get_city_name(loc_1, loc_2)
                        for city in cities:
                            if city != w:
                                depart = city
                        break
            
            #si il n'y a pas d'intention dans la phrase, on passe à la deuxième partie d'analyse, 
            # l'action, on va chercher à savoir si sémantiquement une action est déclaré,
            #  par exemple: je vais à paris. grâce au verbe aller on sait qu'une action va se produire ou se déroule
            if not destination:
                for [w, label] in data:
                    if label == "ACTION":
                        action = True
                        pass
                    if action:
                        if label == "LOC":
                            destination = w
                            cities = get_city_name(loc_1, loc_2)
                            for city in cities:
                                if city != w:
                                    depart = city
                            break
    #Retourne le résultat de l'analyse en faisant une transformation sur les données par la même occasion pour les villes spéciales
    return ["L'" + x if x.lower() in gares_without_starting else x for x in [depart, destination, ",".join(etapes)]]
get_position(text)

['PARIS', 'STRASBOURG', '']

In [26]:
#Process de tests pour calculer les résultats de l'IA
nb_passed, nb_no_passed = 0, 0

for i, data in liste_phrase_test.iterrows():
    sentence = data["phrase"]
    destination = data["destination"]
    depart = data["départ"]
    etapes = data["étapes"]
    
    if [depart.upper(), destination.upper(), etapes.upper()] == get_position(sentence):
        nb_passed += 1
    else:
        print(sentence)
        [data, locs] = get_main_data(sentence)
        #print(locs)
        nb_no_passed += 1

print("Nombre passé: ", nb_passed)
print("Nombre non passé: ", nb_no_passed)
print("Nombre total: ", nb_passed + nb_no_passed)


Je souhaite visiter un musée de Paris, je pars de strasbourg
Je vais visiter un musée de paris, je pars de strasbourg
Je suis à strasbourg et je retourne à paris
Je veux aller à Paris en partant de Strasbourg et en passant par Dijon et Bordeaux
Strasbourg j'irai, de Paris je partirai
Strasbourg j'irai, de Paris je partirai, par Lyon je passerai
En passant par Lyon et bordeaux, j'aimerais partir à strasbourg depuis Paris
En passant par Lyon puis bordeaux, j'aimerais partir à strasbourg depuis Paris
Nombre passé:  28
Nombre non passé:  8
Nombre total:  36


In [14]:
#Permet de développer l'analyse d'une phrase qui ne serait pas passer
text_test = "En passant par Lyon et bordeaux, j'aimerais partir à strasbourg depuis Paris"
text_test = data_treatment(text_test)
doc = nlp(text_test)
print([(w.text, w.pos_) for w in doc])

get_main_data(text_test)

[('en', 'ADP'), ('passant', 'VERB'), ('par', 'ADP'), ('lyon', 'PROPN'), ('et', 'CCONJ'), ('bordeaux', 'ADJ'), ("j'", 'PRON'), ('aimerais', 'VERB'), ('partir', 'VERB'), ('à', 'ADP'), ('strasbourg', 'PROPN'), ('depuis', 'ADP'), ('paris', 'PROPN')]


[[['par', 'ADP'],
  ['LYON', 'LOC'],
  ['BORDEAUX', 'LOC'],
  ['aimer', 'POT_ACTION'],
  ['partir', 'ACTION'],
  ['à', 'ADP'],
  ['STRASBOURG', 'LOC'],
  ['depuis', 'ADP'],
  ['PARIS', 'LOC']],
 [[['par', 'ADP'], ['LYON', 'LOC']],
  [['BORDEAUX', 'LOC']],
  [['à', 'ADP'], ['STRASBOURG', 'LOC']],
  [['depuis', 'ADP'], ['PARIS', 'LOC']]]]